In [31]:
from pymongo import MongoClient
import pprint
import indicoio as ico 
ico.config.api_key = "07ce0e1a82169f11530188d15042c10a"

In [35]:
client = MongoClient('mongodb://localhost:27017/myDB')
db = client['mydb']
wall_posts = db['wall_posts']

analyzer = SentimentIntensityAnalyzer()

for post in wall_posts.find({}):
    
    if vs['pos']> 0.0:
        print("\n\n", "********************************************************************************" )
        print(post['text'])
        print(ico.sentiment(post['text']))
        print( "********************************************************************************" )
   
    



 ********************************************************************************
Игра "Радуга смайлами"♡
Приз- 200💎
1/12- красный
Sad~
0.3789597687407745
********************************************************************************


 ********************************************************************************
Проблема не в отношениях , а в человеке , любит он ( а)  или нет , если человек не испытывает любовь к тебе тогда нет смысла встречаться
0.046698578567995495
********************************************************************************


 ********************************************************************************
Мое открытие на Shazam: XXXTENTACION - Sad!.
0.2554580163940489
********************************************************************************


 ********************************************************************************
🍓Отличная закупка для красавиц. 😻Тушь, румына, тени, помады, маски для лица, кисти и многое-многое-многое другое! 🙀Спешите обн

0.6689040111340575
********************************************************************************


 ********************************************************************************
Broken smile (my all)
————————————————————
[Intro] 
I gotta go right now that's all 
Nothing to it don't expect no call 
Y'all been plotting on me, praying I fall 
I been working, I been giving my all 
I gotta go right now that's all 
Nothing to it don't expect no call 
Y'all been plotting on me, praying I fall 
I been working, I been giving my all 

[Chorus] 
She was the one with the broken smile 
Now that's done, she was the one 
She was the one that was worth my time 
Now that she gone, I know that I'm wrong 
She was the one with the broken smile 
Now that's done, she was the one 
She was the one that was worth my time 
Now that she gone, I know that I'm wrong 

[Verse] 
One more chance to give and that's all I have 
All I have left in my heart 
I've been through enough to know– 
When you're sad and we

0.2672346957368974
********************************************************************************


 ********************************************************************************
Sad
0.01730476618956247
********************************************************************************


 ********************************************************************************
качественный кОнтент
покидайте в комменты зацепившие вас песенки, которые добавляли себе самыми последними
0.01347319705298453
********************************************************************************


 ********************************************************************************
🍐КОЛОНОВИДНАЯ ГРУША!
Миф ради прибыли

Как не купить карликовую грушу по тройной цене рассказывает наш партнер, селекционер, кандидат сельскохозяйственных наук, Качалкин М.В.

👍🏻Купить настоящую вкусную грушу по доступной цене вы можете у нас на сайте: http://zelenii-sad.com/plodovoyadodnye-kultury/grusha/
0.1959605204562313
********

0.513738259142112
********************************************************************************


 ********************************************************************************

0.46901690857727124
********************************************************************************


 ********************************************************************************
We know it is a sad day for all her fans out there but what we also know is that Sridevi is in a better place. We hope and pray for her family. RIP Sridevi.
0.3251921683651478
********************************************************************************


 ********************************************************************************
I recently moved to US about a month ago with my girlfriend to live with her. She recently got kicked out of her family house and I feel so...

👋 Friends! Please give 💓 an advice to the 😔 person in this ☔ situation. Repost this or click here and download app! Perhaps thanks to you there is